<div style="background-color: crimson; padding: 2px; border-radius: 15px; text-align: center;">
    <a class="anchor" id="1"></a>
    <h3 id="#1" style="color: white; font-size: 18px; font-family: 'Poppins', sans-serif; font-weight: bold;">1. Introdução </h3>  
</div> 

[Voltar ao início](#top)

A estratégia adotada para o desenvolvimento do modelo de classificação de reclamações envolveu uma abordagem integrada e iterativa, focada tanto no processamento de dados quanto na modelagem. Esta estratégia pode ser descrita em várias fases-chave, que são detalhadas a seguir:

- **Pré-processamento de Dados Rigoroso**: A primeira fase da estratégia envolveu um pré-processamento de dados abrangente, visando limpar e normalizar os textos das reclamações. Operações como a remoção de URLs, tags HTML, pontuações, números, acentos, e a conversão de todos os caracteres para minúsculas foram cruciais para reduzir a complexidade dos dados. A lematização e a remoção de stopwords específicas do idioma Português ajudaram a focar no conteúdo semântico relevante, minimizando o ruído e as redundâncias nos dados.

- **Transformação dos Textos em Estruturas Numéricas**: Utilizando técnicas de tokenização e padding, os textos foram convertidos em sequências numéricas de comprimento fixo, preparando-os para o processamento pela rede neural. A tokenização foi essencial para transformar o texto em uma forma que a rede neural pudesse interpretar, enquanto o padding garantiu que todas as sequências tivessem o mesmo tamanho, uma necessidade técnica para o treinamento do modelo.

- **Arquitetura de Modelo Avançada**: A escolha de uma rede neural sequencial com camadas de Embedding e LSTM Bidirecional representou uma estratégia focada em capturar a natureza sequencial e os contextos bidirecionais do texto. A camada de Embedding proporcionou uma representação densa e significativa das palavras, enquanto as camadas LSTM Bidirecionais foram capazes de aprender dependências de longo prazo em ambas as direções do texto, aumentando a capacidade de compreensão do modelo.

- **Técnicas de Regularização e Controle de Overfitting**: Para combater o overfitting, foram aplicadas técnicas de regularização L2 nas camadas LSTM, além do uso de dropout para adicionar aleatoriedade no processo de aprendizado, reduzindo a dependência do modelo em características específicas do conjunto de treinamento. O Early Stopping foi implementado para monitorar a acurácia de validação e interromper o treinamento assim que o modelo começasse a mostrar sinais de overfitting, garantindo que o modelo retido fosse aquele com o melhor desempenho geral sem ser excessivamente ajustado aos dados de treinamento.

- **Avaliação e Iteração**: Após o treinamento inicial, o modelo foi rigorosamente avaliado usando métricas como precisão, recall, e F1-score para cada categoria de classificação. Essa avaliação permitiu identificar categorias onde o modelo poderia ser melhorado, guiando ajustes subsequentes no pré-processamento, na arquitetura do modelo, ou nas técnicas de treinamento.

<div style="background-color: crimson; padding: 2px; border-radius: 15px; text-align: center;">
    <a class="anchor" id="2"></a>
    <h3 id="#2" style="color: white; font-size: 18px; font-family: 'Poppins', sans-serif; font-weight: bold;">2. Importar bibliotecas</h3>  
</div> 

[Voltar ao início](#top)

In [1]:
import pandas as pd
import re
import boto3
import string
import spacy
import nltk
import os
import unicodedata
from numpy import argmax
from spacy.lang.pt.stop_words import STOP_WORDS
from io import StringIO
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from dotenv import load_dotenv
from tensorflow.keras.callbacks import EarlyStopping

! python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 24.5 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


<div style="background-color: crimson; padding: 2px; border-radius: 15px; text-align: center;">
    <a class="anchor" id="3"></a>
    <h3 id="#3" style="color: white; font-size: 18px; font-family: 'Poppins', sans-serif; font-weight: bold;">3. Carregar Dataset, Parâmetros e Funções</h3>  
</div> 

[Voltar ao início](#top)

In [6]:
# Load variables from .env
load_dotenv()

# Access environment variables.
aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
bucket_name = os.getenv('BUCKET_NAME')
object_key = os.getenv('OBJECT_KEY')

# Try to access AWS, if not, then load local data.
try:
    s3_client = boto3.client('s3', region_name='us-east-1', 
                         aws_access_key_id=aws_access_key_id, 
                         aws_secret_access_key=aws_secret_access_key)
    csv_obj = s3_client.get_object(Bucket=bucket_name, Key=object_key)
    body = csv_obj['Body']
    csv_string = body.read().decode('utf-8')
    df = pd.read_csv(StringIO(csv_string),sep=';')
    print("Loaded data from S3.")
except Exception as e:
    print(f"Failed to load data from S3 due to {e}. Attempting to load from local directory.")
    local_path = '../data/tickets_reclamacoes_classificados.csv'
    df = pd.read_csv(local_path,sep=';')
    print("Loaded data from local directory.")
    
# Portuguese data for Space.
nlp = spacy.load("pt_core_news_sm")


Failed to load data from S3 due to An error occurred (InvalidAccessKeyId) when calling the GetObject operation: The AWS Access Key Id you provided does not exist in our records.. Attempting to load from local directory.
Loaded data from local directory.


In [4]:
# Create functions.

def normalize_text(text):
    """
    Normalize the input string by converting to lowercase, removing punctuation,
    digits, any special characters not included in the Portuguese alphabet, and
    extra spaces. This function also applies accent removal.
    
    Parameters:
    text (str): The input string to be normalized.
    
    Returns:
    str: The normalized string.
    """
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^a-záéíóúàèìòùâêîôûãõäëïöüç\s]', '', text)
    text = ''.join((c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn'))
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def lemmatize_and_remove_stopwords(text):
    """
    Apply lemmatization to the input string and remove stopwords using the
    spaCy library for the Portuguese language. Only non-stopword tokens are
    lemmatized and concatenated into a single string.
    
    Parameters:
    text (str): The input string to be lemmatized and from which stopwords will be removed.
    
    Returns:
    str: The lemmatized string with stopwords removed.
    """
    doc = nlp(text)
    result = []
    for token in doc:
        if not token.is_stop:
            result.append(token.lemma_)
    return ' '.join(result)

<div style="background-color: crimson; padding: 2px; border-radius: 15px; text-align: center;">
    <a class="anchor" id="4"></a>
    <h3 id="#4" style="color: white; font-size: 18px; font-family: 'Poppins', sans-serif; font-weight: bold;">4. Pré-processamento</h3>  
</div> 

[Voltar ao início](#top)

## 2.1. Normalização dos dados

[Voltar ao início](#top)

In [5]:
# Applies normalizations.
for col in ['descricao_reclamacao', 'categoria']:
    df[f'{col}_norm'] = df[col].apply(normalize_text).apply(lemmatize_and_remove_stopwords)
df.head()

,id_reclamacao,data_abertura,categoria,descricao_reclamacao,descricao_reclamacao_norm,categoria_norm
0,3229299,2019-05-01T12:00:00-05:00,Hipotecas / Empréstimos,"Bom dia, meu nome é xxxx xxxx e agradeço se vo...",dia nome xxxx xxxx agradeco voce puder ajudar ...,hipoteca emprestimos
1,3199379,2019-04-02T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,Atualizei meu cartão xxxx xxxx em xx/xx/2018 e...,atualizei cartao xxxx xxxx informar agente atu...,Cartao credito cartao prepago
2,3233499,2019-05-06T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,O cartão Chase foi relatado em xx/xx/2019. No ...,cartao chase relatar entanto pedido fraudulent...,Cartao credito cartao prepago
3,3180294,2019-03-14T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,"Em xx/xx/2018, enquanto tentava reservar um ti...",tentar reservar ticket xxxx xxxx deparei ofert...,Cartao credito cartao prepago
4,3224980,2019-04-27T12:00:00-05:00,Serviços de conta bancária,"Meu neto me dê cheque por {$ 1600,00} Eu depos...",neto cheque depositar conta chase Fundo limpo ...,servico conta bancario


## 2.2. Separação entre teste e treino
[Voltar ao início](#top)

In [6]:
X = df['descricao_reclamacao_norm']
y = df['categoria_norm']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## 2.3. Stopwords e Vetorização
[Voltar ao início](#top)

- **Definição da Lista de Stopwords em Português**: Retorna uma lista de stopwords palavras em português.

- **CountVectorizer com Restrição de Stopwords e Unigrama**: Converte uma coleção de documentos de texto em uma matriz de contagens de tokens vetorizados em Unigramas e instrui o vetorizador a ignorar as stopwords em portugês.

In [7]:
pt_br_stopwords = nltk.corpus.stopwords.words('portuguese')
vect = CountVectorizer(ngram_range=(1,1), stop_words=pt_br_stopwords)

## 2.4. Tokenização, limite de tamanho e *padding*
[Voltar ao início](#top)

A tokenização divide o texto em unidades menores (tokens), geralmente palavras ou termos. A tokenização é fundamental para a conversão de texto em uma forma que pode ser processada por algoritmos de aprendizado de máquina.

Também limitamos o tamanho do vocabulário para ajudar a manter o modelo gerenciável, reduzindo a complexidade e o risco de *overfitting*.

Como utilizamos um modelo de **Redes Neurais** para nossos dados e esse tipo de modelo geralmente requer entradas de tamanho fixo, aplicamos o *padding* para   garantir que todas as sequências de entrada tenham o mesmo comprimento, preenchendo sequências mais curtas com zeros.

In [8]:
max_words = 20000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_length = 200
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

y_train_encoded = to_categorical(y_train_encoded)
y_test_encoded = to_categorical(y_test_encoded)

vectorizer = TfidfVectorizer(max_features=10000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


<div style="background-color: crimson; padding: 2px; border-radius: 15px; text-align: center;">
    <a class="anchor" id="5"></a>
    <h3 id="#5" style="color: white; font-size: 18px; font-family: 'Poppins', sans-serif; font-weight: bold;">5. Modelo e Validações</h3>  
</div> 

[Voltar ao início](#top)

In [9]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=100, input_length=max_length))
model.add(Bidirectional(LSTM(100, return_sequences=False, dropout=0.5, recurrent_dropout=0.5, kernel_regularizer=l2(0.01))))
model.add(Dense(y_train_encoded.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 100)          2000000   
                                                                 
 bidirectional (Bidirection  (None, 200)               160800    
 al)                                                             
                                                                 
 dense (Dense)               (None, 5)                 1005      
                                                                 
Total params: 2161805 (8.25 MB)
Trainable params: 2161805 (8.25 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=2)
model.fit(X_train_pad, y_train_encoded, epochs=8, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/8
198/198 [==============================] - 99s 493ms/step - loss: 1.7936 - accuracy: 0.4977 - val_loss: 0.9930 - val_accuracy: 0.6264
Epoch 2/8
198/198 [==============================] - 115s 580ms/step - loss: 0.8940 - accuracy: 0.6858 - val_loss: 0.7539 - val_accuracy: 0.7713
Epoch 3/8
198/198 [==============================] - 109s 551ms/step - loss: 0.8565 - accuracy: 0.6945 - val_loss: 0.7845 - val_accuracy: 0.7001
Epoch 4/8
198/198 [==============================] - 111s 560ms/step - loss: 0.6423 - accuracy: 0.8121 - val_loss: 0.6622 - val_accuracy: 0.8086
Epoch 5/8
198/198 [==============================] - 108s 548ms/step - loss: 0.5519 - accuracy: 0.8549 - val_loss: 0.6127 - val_accuracy: 0.8219
Epoch 6/8
198/198 [==============================] - 110s 554ms/step - loss: 0.5017 - accuracy: 0.8661 - val_loss: 0.9543 - val_accuracy: 0.7105
Epoch 7/8
198/198 [==============================] - 107s 541ms/step - loss: 0.4591 - accuracy: 0.8834 - val_loss: 0.5846 - val_acc

In [11]:
y_pred = model.predict(X_test_pad)
y_pred_classes = argmax(y_pred, axis=1)
y_test_classes = argmax(y_test_encoded, axis=1)

print(classification_report(y_test_classes, y_pred_classes, target_names=label_encoder.classes_))

165/165 [==============================] - 11s 64ms/step
                               precision    recall  f1-score   support

                                    0.88      0.70      0.78       549
Cartao credito cartao prepago       0.86      0.87      0.87      1290
         hipoteca emprestimos       0.86      0.87      0.86       922
      roubo relatorio disputa       0.82      0.86      0.84      1204
       servico conta bancario       0.87      0.89      0.88      1303

                     accuracy                           0.85      5268
                    macro avg       0.86      0.84      0.85      5268
                 weighted avg       0.86      0.85      0.85      5268



<div style="background-color: crimson; padding: 2px; border-radius: 15px; text-align: center;">
    <a class="anchor" id="6"></a>
    <h3 id="#6" style="color: white; font-size: 18px; font-family: 'Poppins', sans-serif; font-weight: bold;">6. Conclusão</h3>  
</div> 

[Voltar ao início](#top)

O modelo de classificação desenvolvido demonstrou um desempenho geral robusto, com uma precisão (accuracy) de 85% no conjunto de testes. Isso indica uma capacidade significativa do modelo em identificar corretamente a categoria de reclamações entre as opções disponíveis: Cartão de Crédito/Cartão Pré-pago, Hipoteca/Empréstimos, Roubo/Relatório de Disputa e Serviço/Conta Bancária. As métricas individuais de precisão, recall e F1-score para cada categoria sugerem que o modelo é relativamente equilibrado em termos de desempenho entre as diferentes categorias, embora algumas variações sejam observáveis. Por exemplo, a categoria com a menor recall (0.70) demonstra que há espaço para melhorias na capacidade do modelo de identificar todos os casos relevantes para essa categoria.